In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, lower, regexp_replace
from pyspark.sql.types import IntegerType, DoubleType
import os

# --- 1. Configuração do Spark e Variáveis ---
spark = SparkSession.builder \
    .appName("SilverLayerProcessing") \
    .getOrCreate()

# Caminhos do Data Lake no HDFS
BRONZE_SUICIDE_PATH = 'hdfs://namenode:9870/datalake/bronze/raw_suicidio.json'
BRONZE_DEPRESSION_PATH = 'hdfs://namenode:9870/datalake/bronze/raw_depressao.json'

SILVER_PATH = 'hdfs://namenode:9870/datalake/silver/who_data_silver.parquet'

# Mapeamento de Indicadores
INDICATORS_MAP = {
    'SDGSUICIDE': 'Taxa_Mortalidade_Suicidio',
    'MH_12': 'Prevalencia_Transtorno_Humor'
}

def load_bronze_data(path, indicator_name):
    """Carrega o arquivo JSON bruto do HDFS, aplicando um 'flatten' inicial."""
    print(f"Lendo dados brutos ({indicator_name}) de: {path}")
    
    try:
        df_raw = spark.read.json(path, multiLine=True)
        # Seleciona o array de dados ('value') e explode ele para ter uma linha por registro.
        df_exploded = df_raw.selectExpr("explode(value) as data")
        
        # Seleciona os campos aninhados que nos interessam
        df_selected = df_exploded.select(
            col("data.GHO").alias("GHO_ID"),
            col("data.YEAR").alias("Year"),
            col("data.COUNTRY").alias("Country_Code"),
            col("data.NUMERIC_VALUE").alias("Value")
        ).withColumn("Indicator_Name_Source", lit(indicator_name)) # Para fácil rastreamento
        
        return df_selected
    except Exception as e:
        print(f"ERRO ao carregar dados do HDFS em {path}: {e}")
        return spark.createDataFrame([], schema=None) # Retorna DataFrame vazio em caso de erro

def transform_to_silver(df_suicide, df_depression):
    
    print("Iniciando limpeza e unificação (Camada Silver)...")
    
    # 1. União dos dados de Suicídio e Depressão
    df_combined = df_suicide.unionByName(df_depression)

    # 2. Filtragem e Renomeação (Aplicar o mapeamento)
    
    df_silver = df_combined.withColumn(
        "Indicator_Name", 
        col("GHO_ID").cast("string").replace(INDICATORS_MAP) 
    )
    # Usando .when().otherwise() para mapear:
    from pyspark.sql.functions import when
    df_silver = df_combined.withColumn(
        "Indicator_Name",
        when(col("GHO_ID") == "SDGSUICIDE", INDICATORS_MAP['SDGSUICIDE'])
        .when(col("GHO_ID") == "MH_12", INDICATORS_MAP['MH_12'])
        .otherwise(col("GHO_ID"))
    )

    # 3. Tratamento de Tipos de Dados e Nulos
    df_silver = df_silver.withColumn("Year", col("Year").cast(IntegerType())) \
                         .withColumn("Value", col("Value").cast(DoubleType()))

    # Remove linhas onde o valor da taxa ou o ano é nulo/inválido
    df_silver.dropna(subset=['Value', 'Year'], how='any', inplace=True)
    
    # 4. Seleção final das colunas (limpa)
    df_final = df_silver.select(
        "Country_Code", 
        "Year", 
        "Indicator_Name", 
        "Value"
    )
    
    print(f"Limpeza concluída. Total de linhas na Silver: {df_final.count()}")
    return df_final


def save_silver_data(df_silver, file_path):
    """Salva o DataFrame limpo no formato Parquet no HDFS."""
    if df_silver.count() > 0:
        print(f"Salvando dados limpos (Camada Silver) em: {file_path}")
        # Salvar em formato Parquet
        df_silver.write.mode("overwrite").parquet(file_path)
        print("Dados salvos com sucesso!")


if __name__ == "__main__":
    # Carregar dados brutos
    df_suicide = load_bronze_data(BRONZE_SUICIDE_PATH, "Suicidio")
    df_depression = load_bronze_data(BRONZE_DEPRESSION_PATH, "Depressao")
    
    if df_suicide.count() > 0 and df_depression.count() > 0:
        df_silver = transform_to_silver(df_suicide, df_depression)
        save_silver_data(df_silver, SILVER_PATH)
    else:
        print("Não foi possível processar. Verifique se os arquivos Bronze existem no HDFS.")
    
    spark.stop() # Encerrar a sessão Spark

ModuleNotFoundError: No module named 'pyspark'